<font color='blue' size='5px'/> Implement in Project<font/>

# 1 Packages

In [1]:
!pip install keras-cv-attention-models

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 617.3/617.3 kB 11.6 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.8 MB/s eta 0:00:0000:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 21.12.2 requires cupy-cuda115, which is not installed.
tfx-bsl 1.12.0 requires google-api-python-client<2,>=1.7.11, but you have google-api-python-client 2.82.0 which is incompatible.
tfx-bsl 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
tensorflow-transform 1.12.0 requires pyarrow<7,>=6, but you have pyarrow 5.0.0 which is incompatible.
onnx 1.13.1 requires protobuf<4,>=3.20.2, but you have protobuf 3.19.6 which is incompatible.
apache-beam 2.

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_datasets as tfds 

In [5]:
from kaggle_datasets import KaggleDatasets

In [22]:
from sklearn.model_selection import train_test_split
from keras.layers import Input,Dense, Flatten,Dropout,Conv2D,GlobalAveragePooling2D,MaxPool2D
from keras.models import Sequential
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB7
from keras_cv_attention_models import convnext
from keras_cv_attention_models import mobilenetv3

In [9]:
from multiprocessing import cpu_count
from tqdm.notebook import tqdm

In [10]:

import warnings 
warnings.filterwarnings("ignore", category=FutureWarning)

In [12]:
import os
import PIL
import PIL.Image
import pickle 
import math
import cv2
import datetime
import random
import shutil
import time
import gc
import sys

print(f'Tensorflow Version: {tf.__version__}')
print(f'Python Version: {sys.version}')

Tensorflow Version: 2.11.0
Python Version: 3.7.12 | packaged by conda-forge | (default, Oct 26 2021, 06:08:53) 
[GCC 9.4.0]


# 2 Explore Data

## 2.1 Load Data

A batch size refers to the number of samples that are processed in one forward/backward pass during the training of a neural network. By processing data in batches, the training process becomes more efficient because the weights are updated after processing several samples rather than one at a time. Additionally, batch processing allows the use of larger datasets that may not fit into memory all at once.

 A pipeline typically involves a series of processing steps that transform raw data into a form that is suitable for use in a machine learning model. For example, a pipeline for image classification might include steps for loading the images, resizing or cropping them, applying data augmentation techniques, and normalizing the pixel values
 
 Therefore, while using a batch size is an important consideration for efficient training, it is only one part of a larger pipeline that involves many other processing steps.





$$ Albumentation$$

In [9]:
train_ds=tf.keras.utils.image_dataset_from_directory('/kaggle/input/fake-images-detecttion-dataset/Albumentations_data-20230318T161556Z-001/Albumentations_data/Validation_Data',
                                                     seed=0,
                                                     image_size=(160,160),
                                                     batch_size=16)

Found 5995 files belonging to 2 classes.


In [10]:
valid_ds=tf.keras.utils.image_dataset_from_directory('/kaggle/input/fake-images-detecttion-dataset/Albumentations_data-20230318T161556Z-001/Albumentations_data/Train_Data',
                                                     seed=0,
                                                     image_size=(160,160),
                                                     batch_size=8)

Found 47961 files belonging to 2 classes.


In [11]:
test_ds=tf.keras.utils.image_dataset_from_directory('/kaggle/input/fake-images-detecttion-dataset/Albumentations_data-20230318T161556Z-001/Albumentations_data/Test_Data',
                                                    seed=0,
                                                    image_size=(160,160),
                                                    batch_size=8)

Found 5996 files belonging to 2 classes.


$$ Keras$$

In [13]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/fake-images-detecttion-dataset/Keras_Data-20230318T161559Z-001/Keras_Data/Train_Data',
  seed=0,
  image_size=(160, 160),
  batch_size=16)

Found 47995 files belonging to 2 classes.


In [14]:
valid_ds = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/fake-images-detecttion-dataset/Keras_Data-20230318T161559Z-001/Keras_Data/Validation_Data',
  seed=0,
  image_size=(160, 160),
  batch_size=8)

Found 6000 files belonging to 2 classes.


In [15]:
test_ds = tf.keras.utils.image_dataset_from_directory(
  '/kaggle/input/fake-images-detecttion-dataset/Keras_Data-20230318T161559Z-001/Keras_Data/Test_Data',
  seed=0,
  image_size=(160, 160),
  batch_size=8)

Found 6001 files belonging to 2 classes.


The prefetch method is used to load data into memory in advance of when it is needed during training or validation, which can help improve training speed by reducing I/O wait times.

The buffer_size parameter of the prefetch method specifies the number of samples to prefetch. By setting it to AutoTune, the buffer size is automatically tuned to the available system resources, such as CPU and memory, for optimal performance.


In [16]:
AutoTune=tf.data.AUTOTUNE
train_ds=train_ds.prefetch(buffer_size=AutoTune)
val_ds=valid_ds.prefetch(buffer_size=AutoTune)

# 3 Freature Engineering 

## 3.1 Preprocessing Data

In [38]:
# Seed all random number generators
def seed_everything(seed=0):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    tf.random.set_seed(seed)

seed_everything()

We want to match the images that we have to the same shape images in ImageNet, which the model was trained on so we will use cast and imagenet_utils

- tf.cast is used to cast the input tensor image to a tf.float32 data type. This is typically done to ensure that the input data type is consistent with the data type expected by the subsequent layers of the model.

- tf.keras.applications.imagenet_utils.preprocess_input is used to normalize the image with respect to the ImageNet mean. This preprocessing step is commonly used when working with image datasets, as it helps to ensure that the input data is consistent with the data that the pre-trained models have been trained on.

- The mode parameter is set to 'torch', which specifies that the normalization is performed according to the Torchvision implementation. Specifically, the pixel values of the input image are first scaled between -1 and 1, and then the channel means of the ImageNet dataset are subtracted from the pixel values.

In [19]:
def normalize(image):
# Repeat channels to create 3 channel images required by pretrained ConvNextV2 models
# image = tf.repeat(image, repeats=3, axis=3)
    # Repeat channels to create 3 channel images required by pretrained ConvNextV2 models
    # image=tf.repeat(image,repeats=3,axis=3)
    # Cast to float32
    image=tf.cast(image,tf.float32)
    #Normalize with respect to ImageNet Mean
    image=tf.keras.applications.imagenet_utils.preprocess_input(image,mode='torch')
    return image

Default Strategy in Tensor flow that it workes on CPU and single GPU, so we need to change it to work on two

- A MirroredStrategy object, which is used for distributing training across multiple GPUs.
- Then you are getting the number of replicas (i.e., the number of GPUs) available in the strategy using the num_replicas_in_sync attribute.

In [20]:
STRATEGY=tf.distribute.MirroredStrategy()
N_REPLICAS=STRATEGY.num_replicas_in_sync

In [21]:
N_REPLICAS

2

Done in last Phase

## 3.2 Feature Extraction

During training we will use EfficientNet for that

## 3.3 Dimensionality Reduction

We will use Pooling in Training

# 4 Preprocessing

We will normalize the data, and making sure that the model is using the right amount of GPU using With statement

# 5 Training

In [40]:
N_EPOCHS=3
VERBOSE=1
N_REPLICAS=2
LR_MAX=5e-6 * N_REPLICAS
WD_RATIO=1e-5
N_WARMUP_EPOCHS=0

In [39]:
def get_model():
    # Verify Mixed Policy settings
    # Print('f'compute dtype)
    
    INPUT_SHAPE=(160,160,3)
    
    with STRATEGY.scope():
        # Set seed for deterministic weights intializations
        seed_everything()
        
        # Input, note the names are equal to directory keys in dataset
        image=tf.keras.layers.Input(INPUT_SHAPE, name='image',dtype=tf.uint8)
        
        
        # Nomalize Input
        image_norm=normalize(image)
        
        # CNN Prediciton in range [0,1]
        x = convnext.ConvNeXtTiny(
            input_shape=(160,160,3),
            pretrained='imagenet21k-ft1k',       ##'imagenet21k-ftlk'
            num_classes=0)(image_norm)
        
        
        # Average Pooling BxhxNxC -> BxC
        x=GlobalAveragePooling2D()(x)
        # Dropout 
        x=Dropout(.5)(x) #
        # Output sigmoid
        outputs=Dense(1,activation='sigmoid')(x)
        
        
        ## We will use Adam optimizer for fast learning with weight decap
        optimizer=tfa.optimizers.AdamW(learning_rate=LR_MAX,weight_decay=LR_MAX*WD_RATIO,epsilon=1e-6)
        
        ## Loss
        loss=tf.keras.losses.BinaryCrossentropy(from_logits=False)
        
        
        ## Metrics
        metrics= [tfa.metrics.F1Score(num_classes=1),
        tf.keras.metrics.Precision(),
        tf.keras.metrics.Recall(),
        tf.keras.metrics.AUC(),
        tf.keras.metrics.BinaryAccuracy()]
        
        model=Model(inputs=image,outputs=outputs)
        
        model.compile(optimizer=optimizer,loss=loss,metrics=metrics)
        
        return model


Learning Rate scheduler with logaritimic warmup and cosine decay 
- We want to reduce learning rate as the adam optimizer go on

In [46]:
# Learning rate scheduler with logaritmic warmup and cosine decay
def lrfn(current_step, num_warmup_steps, lr_max, num_cycles=0.50, num_training_steps=N_EPOCHS):
    
    if current_step < num_warmup_steps:
        return lr_max * 0.10 ** (num_warmup_steps - current_step)
    else:
        progress = float(current_step - num_warmup_steps) / float(max(1, num_training_steps - num_warmup_steps))

        return max(0.0, 0.5 * (1.0 + math.cos(math.pi * float(num_cycles) * 2.0 * progress))) * lr_max

Tensorflow learning rate scheuler does not update weight decay , need to do it manually in a custom callback

In [ ]:
# Tensorflow Learning Rate Scheduler does not update weight decay, need to do it manually in a custom callback
class WeightDecayCallback(tf.keras.callbacks.Callback):
    def __init__(self, wd_ratio=WD_RATIO):
        self.step_counter = 0
        self.wd_ratio = wd_ratio
    
    def on_epoch_begin(self, epoch, logs=None):
        model.optimizer.weight_decay = model.optimizer.learning_rate * self.wd_ratio
        print(f'learning rate: {model.optimizer.learning_rate.numpy():.2e}, weight decay: {model.optimizer.weight_decay.numpy():.2e}')

Learning rate for encoder

In [52]:
LR_SCHEDULE = [lrfn(step, num_warmup_steps= N_WARMUP_EPOCHS, lr_max=LR_MAX, num_cycles=.5) for step in range(N_EPOCHS)]
lr_callback=tf.keras.callbacks.LearningRateScheduler(lambda step: LR_SCHEDULE[step],verbose=0)
model=get_model()

>>>> Load pretrained from: /root/.keras/models/convnext_tiny_224_imagenet21k-ft1k.h5


In [53]:
history=model.fit(train_ds,
                  validation_data=valid_ds,
                  epochs=N_EPOCHS,
                  verbose=VERBOSE,
                  callbacks=[lr_callback,
                             WeightDecayCallback()])

learning rate:1.00e-05,weight decay:1.00e-10
Epoch 1/3


2023-03-31 17:45:29.741529: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: Size of values 0 does not match size of permutation 4 @ fanin shape inmodel_4/convnext_tiny/stack1_block2_drop/dropout/SelectV2-2-TransposeNHWCToNCHW-LayoutOptimizer


3000/3000 [==============================] - 617s 194ms/step - loss: 0.7172 - f1_score: 0.6667 - precision_4: 0.7478 - recall_4: 0.7476 - auc_4: 0.8165 - binary_accuracy: 0.7478 - val_loss: 0.2821 - val_f1_score: 0.6667 - val_precision_4: 0.8426 - val_recall_4: 0.9813 - val_auc_4: 0.9767 - val_binary_accuracy: 0.8990 - lr: 1.0000e-05
learning rate:7.50e-06,weight decay:7.50e-11
Epoch 2/3
3000/3000 [==============================] - 535s 178ms/step - loss: 0.2732 - f1_score: 0.6667 - precision_4: 0.9005 - recall_4: 0.9056 - auc_4: 0.9616 - binary_accuracy: 0.9028 - val_loss: 0.1393 - val_f1_score: 0.6667 - val_precision_4: 0.9466 - val_recall_4: 0.9577 - val_auc_4: 0.9879 - val_binary_accuracy: 0.9518 - lr: 7.5000e-06
learning rate:2.50e-06,weight decay:2.50e-11
Epoch 3/3
3000/3000 [==============================] - 573s 191ms/step - loss: 0.1783 - f1_score: 0.6668 - precision_4: 0.9358 - recall_4: 0.9368 - auc_4: 0.9813 - binary_accuracy: 0.9363 - val_loss: 0.1299 - val_f1_score: 0.666

# 6 Prediction

In [54]:
model.save_weights('model.h5')

In [56]:
test_ds= test_ds.prefetch(buffer_size=AutoTune)

In [57]:
model.evaluate(test_ds)

751/751 [==============================] - 53s 70ms/step - loss: 0.1184 - f1_score: 0.6666 - precision_4: 0.9402 - recall_4: 0.9750 - auc_4: 0.9915 - binary_accuracy: 0.9565


[0.11840986460447311,
 array([0.6665926], dtype=float32),
 0.9402121305465698,
 0.9750000238418579,
 0.991520881652832,
 0.9565072655677795]

# 7 Evaluation

In [59]:
y_pred= []
y_test= []

for image_batch, label_batch in test_ds:
    # apppend True Labels
    y_test.append(label_batch)
    # Compute Predictions
    preds=model.predict(image_batch)
    ## Append Preiction labels
    preds[preds<=.5]=0
    preds[preds>.5]=1
    y_pred.append(preds)
    # convert the true and predicted labels into tensors
correct_labels = tf.concat([item for item in y_test], axis = 0)
predicted_labels = tf.concat([item for item in y_pred], axis = 0)

1/1 [==============================] - 0s 172ms/step


In [60]:
from sklearn.metrics import classification_report
classification_report(correct_labels.numpy(), predicted_labels.numpy())

'              precision    recall  f1-score   support\n\n           0       0.97      0.94      0.96      3001\n           1       0.94      0.97      0.96      3000\n\n    accuracy                           0.96      6001\n   macro avg       0.96      0.96      0.96      6001\nweighted avg       0.96      0.96      0.96      6001\n'